In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.special import erf
from matplotlib import gridspec

### Explain updates on the updated rheology from Hirst & Kohlsted 2003.

Still using the Hirth and Kolstedt 03 wet rheology as an example, I showed how to convert from the experimental flow law to the flow law used in aspect. Moreover, I include a discussion about lower mantle rheology. At the end, a whole mantle rheology is defined and shown with a temperature & pressure profile from aspect a model run. With that, 
the related value entries in the prm file is also included. 3h.

For more detail into this method, look at "explain_update_modHK03_rheology.pdf".

#### Main takeaway

A flow law cannot be simple transferred from lab to numerical models.

One need to consider specific question as for whether the type of loading are identical in lab and real earth and whether the relation between the diffusion and dislocation creep satisfies important constraints of the earth upper mantle.
On one hand, A "F" factor is assigned to the flow law to account for differences of loading state between experiments and the real earth.
On the other hand, the values in the flow law need to be tweaked, sometimes to their limit, in order to satisfy the constraints from the earth's upper mantle.


#### Step 1: define the rheology and the flow law we use

Note that the "HK03_wet_mod_diff" and the "HK03_wet_mod_disl" we use here are formulated with Fugacity instead of Coh, so we need to convert it to a formulation with Coh.

In [ ]:
# modified creep laws from Hirth & Kohlstedt 2003
# for detail, refer to magali's explain_update_modHK03_rheology.pdf file
HK03_wet_mod_diff = \
    {
        # "A" : 10**6.9,  # MPa^(-n-r)*um**p/s
        "A" : 7.1768e6,  # MPa^(-n-r)*um**p/s
        "p" : 3.0,
        "r" : 1.0,
        "n" : 1.0,
        "E" : 375e3,
        "V" : 23e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet": 1.0  # I use this to mark this is a wet rheology, so I need to account for V and E for water later.
    }

HK03_wet_mod_disl = \
    {
        "A" : 10**2.65,
        "p" : 0.0,
        "r" : 1.0,
        "n" : 3.5,
        "E" : 520e3,
        "V" : 24e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet" : 1.0
    }

#### Some notes on the value of E and V
In the "wet_mod" rheology, I use the values in the middle (e.g. for wet diffusion, E is 375 +- 50 KJ / mol).
This rheology has issue some of the constraints from Earth's mantle, which we will show later.

In the "wet_mod1" rheology, I choose the limit values instead.

In [ ]:
# a variation to the previous flow law
# I bring the values to the limit of the range
# for detail, refer to magali's explain_update_modHK03_rheology.pdf file
HK03_wet_mod1_diff = \
    {
        # "A" : 10**6.9,  # MPa^(-n-r)*um**p/s
        "A" : 7.1768e6,  # MPa^(-n-r)*um**p/s
        "p" : 3.0,
        "r" : 1.0,
        "n" : 1.0,
        "E" : 375e3 - 40e3,
        "V" : 23e-6 -5.5e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet": 1.0  # I use this to mark this is a wet rheology, so I need to account for V and E for water later.
    }


HK03_wet_mod1_disl = \
    {
        "A" : 10**2.65,
        "p" : 0.0,
        "r" : 1.0,
        "n" : 3.5,
        "E" : 520e3 + 20e3,
        "V" : 24e-6,
        "d" : 1e4,
        "Coh" : 1000.0,
        "wet" : 1.0
    }

# creep law of water
water_creep = \
    {
        "A" : 87.75,             # H/(10^6*Si)/MPa
        "E" : 50e3,                     # J/mol +/-2e3
        "V" : 10.6e-6                     # m^3/mol+/-1
    }


##### Form of the flow law

$\eta_{df, ds} = F \left(\frac{d^p}{A C^r_{OH}} \right)\dot{\epsilon_E}^{\frac{1-n}{n}}exp\left(\frac{E+P_{lc}V}{nRT}\right)$

Here F is a effective coefficient to account for the differences in the conditions in experiments.

In [ ]:
def CreepRheology(creep_type, strain_rate, P, T, d=None, Coh=None, **kwargs):
    """
    def CreepRheology(creep_type, strain_rate, P, T, d, Coh):

    Calculate viscosity by flow law in form of (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T))
    Units:
     - P: Pa
     - T: K
     - d: mu m
     - Coh: H / 10^6 Si
     - Return value: Pa*s
    Pay attention to pass in the right value, this custom is inherited
    """
    R = 8.314
    A = creep_type['A']
    p = creep_type['p']
    r = creep_type['r']
    n = creep_type['n']
    E = creep_type['E']
    V = creep_type['V']
    # compute value of F(pre factor)
    use_effective_strain_rate = kwargs.get('use_effective_strain_rate', False)
    if use_effective_strain_rate:
        F = 1 / (2**((n-1)/n)*3**((n+2)/2/n))
    else:
        F = 1.0

    if d is None:
        d = creep_type['d']
    if Coh is None:
        Coh = creep_type['Coh']
    # calculate B
    B = A * d**(-p) * Coh**r
    eta = 1/2.0 * F * (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T)) * 1e6

    return eta


def harmonic_average(*args):
    '''
    Harmonic averaging of entries
    Inputs:
        args (list): the averaging will be performed on
                    args[0], args[1], args[2] ...
    '''
    if type(args[0]) == float:
        aver = 0.0
        for arg in args:
            assert(abs(arg) > 1e-15)
            aver += 1.0 / arg
            aver = len(args) / aver
    elif type(args[0]) == np.ndarray:
        length = len(args)
        _size = args[0].size
        aver = np.zeros(_size)
        for arg in args:
            assert(arg.size == _size)
            assert(min(abs(arg)) > 1e-15)
            aver += 1.0 / arg
        aver = length / aver
    else:
        raise TypeError("harmonic_average: inputs must by float or ndarray")
    return aver

#### Step 2: derive a rheology with Coh from the original modified rheology with fugacity
For more detail, see "explain_update_modHK03_rheology.pdf", equation (4) and (5)
The equation we use is:

$A^* = A / A^r_{H_2O}, V^* = V - rV_{H_2O}, E = E - rE_{H_2O} $

In [ ]:
def substitute_fugacity_with_coh(creep, water_creep):
    '''
     
    '''
    new_creep = creep.copy()
    new_creep['A'] = creep['A'] / (water_creep['A'] ** creep['r'])
    new_creep['V'] = creep['V'] - water_creep['V'] * creep['r']
    new_creep['E'] = creep['E'] - water_creep['E'] * creep['r']
    return new_creep

HK03_wet_mod_diff_eff = substitute_fugacity_with_coh(HK03_wet_mod_diff, water_creep)
HK03_wet_mod_disl_eff = substitute_fugacity_with_coh(HK03_wet_mod_disl, water_creep)
HK03_wet_mod1_diff_eff = substitute_fugacity_with_coh(HK03_wet_mod1_diff, water_creep)
HK03_wet_mod1_disl_eff = substitute_fugacity_with_coh(HK03_wet_mod1_disl, water_creep)
print(HK03_wet_mod1_diff_eff)
print(HK03_wet_mod1_disl_eff)  # debug

#### Step 3 we use a pressure and a temperature profile of the upper mantle to derive the result

With this plot, it's clear that the dislocation creep of the 'wet_mod' rheology is too strong.
On ther other hand, constraints from seismic anisotropy indicates dislocation creep in most of the shallow upper mantle, with less anisotropy in the depper upper mantle (todo: reference.)
In turn, the diffusion creep and the dislocation creep show be on the same order. 
This is the task took by the 'wet_mod1' rheology

Note that even with the 'wet_mod1' rheology 1e-15 is still not enough for dislocation creep to be the dominant component.
But this may be surfice to have a dislocation dominant upper mantle (todo: include a plot from model)

In [ ]:
year = 365 * 24 * 3600.0

def pressure_from_lithostatic(z,Tad):
    '''
    A lithostatic pressure profile
    Inputs:
	z (float) - depth in m
	Tad (float) - adiabatic temperature
    '''
    # Density Profile
    refrho = 3300  # kg/m^3
    refT = 1673        # K
    alpha = 3.1e-5  # 1/K
    g = 9.81 # m/s^2
    density = refrho*(1-alpha*(Tad-refT))

    # start loop at 1 because P[0] = 0
    dz = z[1]-z[0]
    P = np.zeros(np.size(z))
    for i in range(1, np.size(z)):
        P[i] = P[i-1] + 0.5*(density[i]+density[i-1])*g*dz
    return P


def temperature_halfspace(z, t, **kwargs):
    '''
    temperature from a half-space cooling model
    Inputs:
	z (float) - depth (m)
	t - age (s)
    kwargs (dict):
        Tm (float) - mantle temperature
    '''
    # Physical constants
    kappa = 1e-6  # thermal diffusivity (m^2/s)
    T_s = 273  # surface temperature (K)
    T_m = kwargs.get("Tm", 1673) # mantle temperature (K)
    T = T_s + (T_m - T_s)*erf(z/(2*np.sqrt(kappa*t)))
    return T

strain_rate = 1e-15
zs = np.arange(0.0, 660.0, 1.0) * 1e3
age = 80e6 * year
Tad_surface = 1673.0 
mad = 0.5715  # deg/km, approximating adiabat to match T(CMB) in Aspect
Tads = Tad_surface + mad*(zs/1e3)
Ts = np.zeros(zs.shape)  # temperature, combine HS model with adiabat
for i in range(zs.size):
    z = zs[i]
    Tad = Tads[i]
    Ts[i] = temperature_halfspace(z, age, Tm = Tad)
Ps = pressure_from_lithostatic(zs, Ts)  # pressure, using the computed temperature
# compute viscosities
etas_disl = CreepRheology(HK03_wet_mod_disl_eff, strain_rate, Ps, Ts, use_effective_strain_rate=True)
etas_diff = CreepRheology(HK03_wet_mod_diff_eff, strain_rate, Ps, Ts, use_effective_strain_rate=True)
etas_comp = harmonic_average(etas_diff, etas_disl) 
# plot figures
fig = plt.figure(figsize=(5, 20))
ax = fig.add_subplot(4, 1, 1)  # plot pressure
ax.plot(Ps/1e9, zs/1e3, 'b')
ax.set_xlabel("Pressure (GPa)")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
ax = fig.add_subplot(4, 1, 2)  # plot temperature
ax.plot(Ts, zs/1e3, 'r')
ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
ax = fig.add_subplot(4, 1, 3)  # plot viscosity
# plot1: wet_mod rheology
ax.semilogx(etas_disl, zs/1e3, 'g', label="dislocation creep (wet_mod)")
ax.semilogx(etas_diff, zs/1e3, 'k', label="diffusion creep (wet_mod)")
ax.invert_yaxis()
ax.set_xlim([1e18, 1e25])
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.legend()
# plot1: use wet_mod1 and do this again
etas_disl1 = CreepRheology(HK03_wet_mod1_disl_eff, strain_rate, Ps, Ts, use_effective_strain_rate=True)
etas_diff1 = CreepRheology(HK03_wet_mod1_diff_eff, strain_rate, Ps, Ts, use_effective_strain_rate=True)
etas_comp1 = harmonic_average(etas_diff1, etas_disl1)
ax = fig.add_subplot(4, 1, 4)  # plot viscosity
ax.semilogx(etas_disl1, zs/1e3, 'g', label="dislocation creep (wet_mod1)")
ax.semilogx(etas_diff1, zs/1e3, 'k', label="diffusion creep (wet_mod1)")
ax.semilogx(etas_comp1, zs/1e3, 'r', label="composite rheology (wet_mod1)")
ax.invert_yaxis()
ax.set_xlim([1e18, 1e25])
ax.set_xlabel("viscosity (Pa*s))")
ax.set_ylabel("Depth (km)")
ax.legend()